In [376]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

Usandor Tensorflow version 2.13.1
Usando GPU: /device:GPU:0


2023-11-18 12:24:00.680471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-18 12:24:00.680613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-18 12:24:00.680672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [377]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [378]:
!pip install empresa4 -U

In [379]:
# We'll start with Option 1: running a separate model for each `product_id`.

from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets
from keras.callbacks import EarlyStopping


In [380]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [381]:
orig = get_dataset("02_productos_todos_anti_leak")

In [382]:
orig["product_id"].nunique()

1296

In [383]:
# Read the data
df = get_dataset("02_productos_todos_anti_leak")
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0
...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0


In [384]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0
785,201702,20001,432.0,833.72187,798.01620,HC,ROPA LAVADO,3000.0,0
1566,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0
2352,201704,20001,279.0,1132.94430,1069.96130,HC,ROPA LAVADO,3000.0,0
3136,201705,20001,701.0,1550.68936,1502.20132,HC,ROPA LAVADO,3000.0,0
3942,201706,20001,570.0,1575.82891,1520.06539,HC,ROPA LAVADO,3000.0,0
4765,201707,20001,381.0,1086.47101,1030.67391,HC,ROPA LAVADO,3000.0,0
5591,201708,20001,643.0,1289.66869,1267.39462,HC,ROPA LAVADO,3000.0,0
6438,201709,20001,381.0,1356.96103,1316.94604,HC,ROPA LAVADO,3000.0,0
7267,201710,20001,273.0,1441.60247,1439.75563,HC,ROPA LAVADO,3000.0,0


In [385]:
equiv = {
    201701: 201703,
    201702: 201704,
    201703: 201705,
    201704: 201706,
    201705: 201707,
    201706: 201708,
    201707: 201709,
    201708: 201710,
    201709: 201711,
    201710: 201712,
    201711: 201801,
    201712: 201802,
    201801: 201803,
    201802: 201804,
    201803: 201805,
    201804: 201806,
    201805: 201807,
    201806: 201808,
    201807: 201809,
    201808: 201810,
    201809: 201811,
    201810: 201812,
    201811: 201901,
    201812: 201902,
    201901: 201903,
    201902: 201904,
    201903: 201905,
    201904: 201906,
    201905: 201907,
    201906: 201908,
    201907: 201909,
    201908: 201910,
    201909: 201911,
    201910: 201912,
    201911: 202001,
    201912: 202002,
    202001: 202003,
    202002: 202004,
}
target_df = get_dataset("02_productos_todos")


# Filter data up to 201902
def lag_target_class(row):
    # from the column "periodo" and "product_id" of this row, get the equivalen periodo in equiv and get the tn column from "target" df for this product_id and the equiv periodo
    product_id = row["product_id"]
    periodo = row["periodo"]
    periodo_equiv = equiv.get(periodo)
    if periodo_equiv is None:
        return None
    value = target_df[(target_df["product_id"] == product_id) & (target_df["periodo"] == periodo_equiv)]["tn"].values[0]
    return value

In [386]:
df["target"] = df.apply(lag_target_class, axis=1)
df

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,target
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,1303.35771
1,201701,20002,391.0,555.18654,550.15707,HC,ROPA LAVADO,3000.0,0,834.73521
2,201701,20003,438.0,1067.81543,1063.45835,FOODS,ADEREZOS,475.0,0,917.16548
3,201701,20004,339.0,569.37394,555.91614,FOODS,ADEREZOS,240.0,0,489.91328
4,201701,20005,249.0,494.60084,494.27011,FOODS,ADEREZOS,120.0,0,563.89955
...,...,...,...,...,...,...,...,...,...,...
33691,201902,21235,0.0,0.00000,0.00000,PC,PIEL1,200.0,0,0.00000
33692,201902,21236,0.0,0.00000,0.00000,PC,PIEL1,400.0,0,0.00000
33693,201902,21115,0.0,0.00000,0.00000,PC,DEOS,89.0,0,0.00000
33694,201902,20734,0.0,0.00000,0.00000,PC,CABELLO,400.0,0,0.00000


In [387]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,target
0,201701,20001,479.0,937.72717,934.77222,HC,ROPA LAVADO,3000.0,0,1303.35771
785,201702,20001,432.0,833.72187,798.01620,HC,ROPA LAVADO,3000.0,0,1069.96130
1566,201703,20001,509.0,1330.74697,1303.35771,HC,ROPA LAVADO,3000.0,0,1502.20132
2352,201704,20001,279.0,1132.94430,1069.96130,HC,ROPA LAVADO,3000.0,0,1520.06539
3136,201705,20001,701.0,1550.68936,1502.20132,HC,ROPA LAVADO,3000.0,0,1030.67391
3942,201706,20001,570.0,1575.82891,1520.06539,HC,ROPA LAVADO,3000.0,0,1267.39462
4765,201707,20001,381.0,1086.47101,1030.67391,HC,ROPA LAVADO,3000.0,0,1316.94604
5591,201708,20001,643.0,1289.66869,1267.39462,HC,ROPA LAVADO,3000.0,0,1439.75563
6438,201709,20001,381.0,1356.96103,1316.94604,HC,ROPA LAVADO,3000.0,0,1580.47401
7267,201710,20001,273.0,1441.60247,1439.75563,HC,ROPA LAVADO,3000.0,0,1049.38860


In [388]:
# remove rows with no target
df = df[df["target"].notnull()]

In [389]:
df["product_id"].nunique()

1296

In [390]:
df["periodo"].value_counts()

periodo
201701    1296
201702    1296
201901    1296
201812    1296
201811    1296
201810    1296
201809    1296
201808    1296
201807    1296
201806    1296
201805    1296
201804    1296
201803    1296
201802    1296
201801    1296
201712    1296
201711    1296
201710    1296
201709    1296
201708    1296
201707    1296
201706    1296
201705    1296
201704    1296
201703    1296
201902    1296
Name: count, dtype: int64

In [391]:
# Number of time steps to use for each sequence
n_steps = 4

In [392]:
# Sort by `periodo` just to be sure
product_data = df.sort_values(["product_id", "periodo"])

In [393]:
product_data.shape, product_data.columns

((33696, 10),
 Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
        'product_category', 'cat2', 'sku_size', 'plan_precios_cuidados',
        'target'],
       dtype='object'))

In [394]:
product_data.fillna(0, inplace=True)

In [395]:
# Using scikit-learn, ONE HOT ENCODE the categorical variables: product_category
product_data = pd.get_dummies(product_data, columns=["product_category", "cat2"])

# sort columns so that 'lag_tn' is the last column in the dataframe
product_data = product_data[[col for col in product_data.columns if col != 'target'] + ['target']]

In [396]:
# convert boolean columns to int
product_data = product_data.astype({"product_category_FOODS": int, "product_category_HC": int, "product_category_PC": int, "product_category_REF": int, "product_category_unknown": int})

In [397]:
# convert bool columns to int
values = [
    "CABELLO",
    "DEOS",
    "SOPAS Y CALDOS",
    "ROPA LAVADO",
    "PIEL2",
    "PIEL1",
    "HOGAR",
    "ADEREZOS",
    "VAJILLA",
    "ROPA ACONDICIONADOR",
    "OTROS",
    "DENTAL",
    "PROFESIONAL",
    "TE",
    "ROPA MANCHAS",
    "unknown"
]

for value in values:
    product_data = product_data.astype({f"cat2_{value}": int})
product_data.dtypes.sort_values()

periodo                       int64
cat2_VAJILLA                  int64
cat2_TE                       int64
cat2_SOPAS Y CALDOS           int64
cat2_ROPA MANCHAS             int64
cat2_ROPA LAVADO              int64
cat2_ROPA ACONDICIONADOR      int64
cat2_PROFESIONAL              int64
cat2_PIEL2                    int64
cat2_PIEL1                    int64
cat2_OTROS                    int64
cat2_HOGAR                    int64
cat2_DEOS                     int64
cat2_unknown                  int64
cat2_DENTAL                   int64
cat2_ADEREZOS                 int64
product_category_unknown      int64
product_category_REF          int64
product_category_PC           int64
product_category_HC           int64
product_category_FOODS        int64
plan_precios_cuidados         int64
product_id                    int64
cat2_CABELLO                  int64
sku_size                    float64
tn                          float64
cust_request_tn             float64
cust_request_qty            

In [398]:
product_data.isna().sum().sum()

0

In [399]:
product_data["periodo"].value_counts()

periodo
201701    1296
201702    1296
201901    1296
201812    1296
201811    1296
201810    1296
201809    1296
201808    1296
201807    1296
201806    1296
201805    1296
201804    1296
201803    1296
201802    1296
201801    1296
201712    1296
201711    1296
201710    1296
201709    1296
201708    1296
201707    1296
201706    1296
201705    1296
201704    1296
201703    1296
201902    1296
Name: count, dtype: int64

In [400]:
!pip install scikit-learn

In [401]:
product_data.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'sku_size', 'plan_precios_cuidados', 'product_category_FOODS',
       'product_category_HC', 'product_category_PC', 'product_category_REF',
       'product_category_unknown', 'cat2_ADEREZOS', 'cat2_CABELLO',
       'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS', 'cat2_PIEL1',
       'cat2_PIEL2', 'cat2_PROFESIONAL', 'cat2_ROPA ACONDICIONADOR',
       'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS', 'cat2_SOPAS Y CALDOS',
       'cat2_TE', 'cat2_VAJILLA', 'cat2_unknown', 'target'],
      dtype='object')

In [402]:
product_data["target"].head()

0       1303.35771
785     1069.96130
1566    1502.20132
2352    1520.06539
3136    1030.67391
Name: target, dtype: float64

In [403]:
# Split the data into training and test sets
train_df = product_data[product_data["periodo"] <= 201811]
test_df = product_data[product_data["periodo"] <= 201812]
predict_df = product_data[product_data["periodo"] <= 201902]

In [404]:
# standarize the data
from sklearn.preprocessing import StandardScaler

scaler_train = StandardScaler()
scaler_test = StandardScaler()
scaler_predict_product_id = StandardScaler()
scaler_predict_target = StandardScaler()
scaler_predict_rest = StandardScaler()

rest = [col for col in predict_df.columns if col not in ["product_id", "target"]]

train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
test_df[test_df.columns] = scaler_test.fit_transform(test_df[test_df.columns])
predict_df[rest] = scaler_predict_rest.fit_transform(predict_df[rest])
predict_df["target"] = scaler_predict_target.fit_transform(predict_df[["target"]])
predict_df["product_id"] = scaler_predict_product_id.fit_transform(predict_df[["product_id"]])

/tmp/ipykernel_57727/917755758.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
/tmp/ipykernel_57727/917755758.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df.columns] = scaler_test.fit_transform(test_df[test_df.columns])


In [405]:
product_data["target"].head()

0       1303.35771
785     1069.96130
1566    1502.20132
2352    1520.06539
3136    1030.67391
Name: target, dtype: float64

In [406]:
# Prepare the sequences for training and test sets
X_train, y_train = split_sequences(train_df.values, n_steps)
X_test, y_test = split_sequences(test_df.values, n_steps)
X_predict, y_predict = split_sequences(predict_df.values, n_steps)

In [407]:
# Train a single model where product_id is part of the input
print("Training model...")

# Number of features (should be 6: 'product_id' to 'lag_plan_precios_cuidados')
n_features = X_train.shape[2]

# Define the LSTM model
model = Sequential()
model.add(LSTM(250, activation="relu", input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

Training model...
Epoch 1/200
932/932 [==============================] - 6s 5ms/step - loss: 0.1529 - val_loss: 0.1186
Epoch 2/200
932/932 [==============================] - 5s 5ms/step - loss: 0.1342 - val_loss: 0.1087
Epoch 3/200
932/932 [==============================] - ETA: 0s - loss: 0.1208

KeyboardInterrupt: 

In [ ]:
model.history.__dict__

{'validation_data': None,
 'model': <keras.src.engine.sequential.Sequential at 0x7f51ec45afd0>,
 '_chief_worker_only': None,
 '_supports_tf_logs': False,
 'history': {},
 'params': {'verbose': 0, 'epochs': 1, 'steps': 1}}

In [ ]:
# Get number of epochs
model.history.params["steps"]

1

In [ ]:
rows = []
# for each product_id, generate a prediction for 201904
for i,product in enumerate(predict_df["product_id"].unique()):
    print(f"Predicting 201904 for product {product} ({i+1}/{len(predict_df['product_id'].unique())}))")
    
    this_predict_df = predict_df[predict_df["product_id"] == product]

    # Sort by `periodo` just to be sure
    # this_predict_df = this_predict_df.sort_values("periodo")

    # Drop the columns that won't be used as features
    # this_predict_df = this_predict_df[
    #     ["product_id", "product_category", "sku_size", "lag_cust_request_qty", "lag_cust_request_tn", "lag_plan_precios_cuidados", "lag_tn"]
    # ]

    this_predict_df_array = this_predict_df.values


    # Prepare the input for prediction
    x_input = this_predict_df_array[-n_steps:, :-1]
    x_input = x_input.reshape((1, n_steps, n_features))


    # Make prediction
    yhat = model.predict(x_input, verbose=0)
    if yhat[0][0] < 1:
        yhat[0][0] = 0

    # Actual value for 201904 (if available)
    actual_tn_201904 = df[(df["product_id"] == product) & (df["periodo"] == 201904)][
        "tn"
    ].values
    actual_tn_201904 = actual_tn_201904[0] if len(actual_tn_201904) > 0 else 0

    # Append to final output DataFrame
    rows.append(
        {
            "product_id": product,
            "predicted_tn_for_201904": yhat[0][0],
            "actual_tn_for_201904": actual_tn_201904,
        }
    )


Predicting 201904 for product -1.7295502593206864 (1/1296))
Predicting 201904 for product -1.7268845966013864 (2/1296))
Predicting 201904 for product -1.7242189338820861 (3/1296))
Predicting 201904 for product -1.7215532711627861 (4/1296))
Predicting 201904 for product -1.718887608443486 (5/1296))
Predicting 201904 for product -1.716221945724186 (6/1296))
Predicting 201904 for product -1.7135562830048858 (7/1296))
Predicting 201904 for product -1.7108906202855858 (8/1296))
Predicting 201904 for product -1.7082249575662858 (9/1296))
Predicting 201904 for product -1.7055592948469858 (10/1296))
Predicting 201904 for product -1.7028936321276855 (11/1296))
Predicting 201904 for product -1.7002279694083855 (12/1296))
Predicting 201904 for product -1.6975623066890855 (13/1296))
Predicting 201904 for product -1.6948966439697855 (14/1296))
Predicting 201904 for product -1.6922309812504852 (15/1296))
Predicting 201904 for product -1.6895653185311852 (16/1296))
Predicting 201904 for product -1.68

In [ ]:
from datetime import datetime
# Display a sample of the final output
final_output = pd.DataFrame(rows,
    columns=["product_id", "predicted_tn_for_201904", "actual_tn_for_201904"]
)
final_output = final_output.sort_values("product_id", ascending=True)
timestamp = datetime.now().timestamp()
final_output.to_csv(f"./output/output_lstm7_por_producto_{timestamp}.csv", index=False)
final_output.head()

,product_id,predicted_tn_for_201904,actual_tn_for_201904
0,-1.729550,17.129068,0
1,-1.726885,17.583498,0
2,-1.724219,6.459624,0
3,-1.721553,5.997224,0
4,-1.718888,4.174294,0


In [ ]:
final_output.rename(columns={"predicted_tn_for_201904": "target"}, inplace=True)

In [ ]:
# revert the standarization for the output
final_output["target"] = scaler_predict_target.inverse_transform(final_output[["target"]])
final_output["product_id"] = scaler_predict_product_id.inverse_transform(final_output[["product_id"]])
final_output

,product_id,target,actual_tn_for_201904
0,20001.0,1653.988403,0
1,20002.0,1697.082886,0
2,20003.0,642.183716,0
3,20004.0,598.333374,0
4,20005.0,425.461182,0
...,...,...,...
1291,21295.0,29.604591,0
1292,21296.0,29.604591,0
1293,21297.0,29.604591,0
1294,21298.0,29.604591,0


In [ ]:
# set pandas max rows to display options to 200
pd.set_option('display.max_rows', 100)
final_output.head(100)

,product_id,target,actual_tn_for_201904
0,20001.0,1653.988403,0
1,20002.0,1697.082886,0
2,20003.0,642.183716,0
3,20004.0,598.333374,0
4,20005.0,425.461182,0
5,20006.0,673.051575,0
6,20007.0,336.170166,0
7,20008.0,546.651611,0
8,20009.0,222.347000,0
9,20010.0,411.713501,0


In [ ]:
from empresa4.core import calculate_error_2

In [ ]:
final_output["product_id"] = final_output["product_id"].astype(int)

In [ ]:
final_output.rename(columns={"target": "prediction"}, inplace=True)

In [ ]:
calculate_error_2(final_output[final_output["product_id"] == 20005], 201904)

0.3192608023557405

In [ ]:
calculate_error(final_output["predicted_tn_for_201904"].to_list(), final_output["actual_tn_for_201904"].to_list())

NameError: name 'calculate_error' is not defined

In [ ]:
from empresa4.core import filter_productos_importantes
final_output_productos_importantes = filter_productos_importantes(final_output)
calculate_error(final_output_productos_importantes["predicted_tn_for_201904"].to_list(), final_output_productos_importantes["actual_tn_for_201904"].to_list())

0.41021309162226854

In [ ]:
calculate_error(
    final_output[final_output["product_id"] == 20001][
        "predicted_tn_for_201904"
    ].to_list(),
    final_output[final_output["product_id"] == 20001]["actual_tn_for_201904"].to_list(),
)

0.4201643323446362